In [1]:
#libraries

import statistics

import salem
import warnings

import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cfeature

from plotnine import *
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize, LogNorm, LinearSegmentedColormap
import matplotlib
import cmocean
import seaborn as sns
import folium

import numpy as np
import pandas as pd
import xarray as xr
from scipy import stats
import datetime as dt
from shapely.geometry import Point, Polygon


import glob

In [2]:
# load shapefile with all MPAs
ca_mpas = gpd.GeoDataFrame.from_file("../MPA_dataview_prototype/Data/Shapefiles/mpas/all_mpas/ca_all_mpas.shp")

mpa_lobos_smr_shape = ca_mpas[ca_mpas['MPA_NAME'] == "Point Lobos SMR"]

mpa_buchon_smr_shape = ca_mpas[ca_mpas['MPA_NAME'] == "Point Buchon SMR"]
